# Imports

In [105]:
from pathlib import Path

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup as bs

# Data

In [106]:
posting_dir = Path('data/html_job_postings/')

In [107]:
posting_list = []

for file in posting_dir.glob('*.html'):
    posting_list.append([file, file.read_text()])

In [108]:
posting_list[0]

[PosixPath('data/html_job_postings/1e92960a19ffdd34_fccid.html'),
 '<html><head><title>Quantitative Analyst - Boston, MA 02116</title></head>\n<body><h2>Quantitative Analyst - Boston, MA 02116</h2>\n<p>Quantitative Analyst (State Street Bank and Trust Company; Boston, MA): The Quantitative Analyst will be part of State Street Treasury’s Treasury Quantitative Analytics (TQA) group. TQA is responsible for developing/implementing/monitoring advanced financial models that are used in company’s capital management, liquidity management, investment portfolio construction, and balance sheet optimization. The group is accountable for in-depth understanding, modeling, and representation of the complex interaction of global markets, customer behaviors, and regulatory oversights to create a view of risk/revenue opportunities and exposures to the investment committee, Board of Directors, senior management, and regulatory agencies. The Quantitative Analyst role is a key contributor to the realizatio

# Data Extraction

In [109]:
def extract_data(html_content):
    
    soup = bs(html_content)

    title_text = soup.find('title').text
    body_text = soup.find('body').text

    bullets = soup.find_all('li')
    bullets_text = []
    for b in bullets:
        bullets_text.append(b.text.strip())
    
    return title_text, body_text, bullets_text

In [110]:
posting_df = pd.DataFrame(columns=['title', 'body', 'bullets'])

for _, html in posting_list:

    title, body, bullets = extract_data(html)
    posting_df = posting_df.append(pd.Series([title, body, bullets], 
                                             index=posting_df.columns), 
                                   ignore_index=True)
                                            

In [111]:
posting_df.head()

,title,body,bullets
0,"Quantitative Analyst - Boston, MA 02116","Quantitative Analyst - Boston, MA 02116\nQuant...",[]
1,"Data Scientist - Mountain View, CA","Data Scientist - Mountain View, CA\nGroundTrut...","[Help senior members of the team to explore, d..."
2,"Data Scientist - Seattle, WA","Data Scientist - Seattle, WA\nA Bachelor or Ma...",[A Bachelor or Masters Degree in a highly quan...
3,Senior Natural Language Processing (NLP) Engin...,Senior Natural Language Processing (NLP) Engin...,[Join a small team creating a proprietary NLU ...
4,"FLEXO FOLDER GLUER OPER - McClellan, CA - McCl...","FLEXO FOLDER GLUER OPER - McClellan, CA - McCl...",[]


In [112]:
posting_df.shape

(1337, 3)

## Extract postings related to 'Data Science' and 'ML'

In [113]:
posting_df = posting_df.iloc[np.where(posting_df.title.str.contains('(data science)|(data scientist)|(machine learning)', 
                                                       case=False))]

In [114]:
posting_df.shape

(547, 3)

## Dropping duplicates

In [120]:
posting_df['bullets'] = posting_df['bullets'].apply(tuple, 1)

In [122]:
posting_df['bullets'].head(n=1).values

array([('Help senior members of the team to explore, develop, productionize and optimize machine learning algorithms and pipelines.', 'Use Hadoop, Spark and Amazon Athena on a daily basis to explore our petabytes of data.', 'Dive deep into our rich set of location data, derive insights and build product prototypes.', 'Collaborate with peer data scientists, engineers and product managers closely.', 'Master degree in Computer Science, Statistics, Mathematics, Engineering; PhD is a plus.', 'Has experience in Statistics, Machine Learning.', 'Fluency in Python.', 'Significant experience with SQL; some relational databases or NoSQL data stores.', 'Familiarity with open source machine learning libraries such as scikit-learn and Spark MLlib.', 'Has experience in Amazon Web Services is a plus.', 'Excellent communication skills.')],
      dtype=object)

In [124]:
posting_df = posting_df.drop_duplicates()

In [125]:
posting_df.shape

(541, 3)

# Persisting Data

In [127]:
posting_df.to_csv('data/postings.csv')